## 練習時間
參考 Day 12 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [1]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

### 之前做過的處理

In [2]:
# 設定 data_path
dir_data = './data/'
f_app_train = os.path.join(dir_data, 'application_train.csv')
f_app_test = os.path.join(dir_data, 'application_test.csv')

app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
# also apply to testing dataset
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

# absolute the value of DAYS_BIRTH
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_test['DAYS_BIRTH'] = abs(app_test['DAYS_BIRTH'])

In [9]:
# app_train['AMT_INCOME_TOTAL_SORTED']=sorted(app_train['AMT_INCOME_TOTAL'])

In [14]:
app_train['INCOME_EQUAL_WIDTH_GROUP'] = pd.cut(app_train['AMT_INCOME_TOTAL'], 4)

In [15]:
app_train['INCOME_EQUAL_WIDTH_GROUP'].value_counts()
# 每個 bin 的值的範圍大小都是一樣的

(-91324.35, 29269237.5]      307510
(87756412.5, 117000000.0]         1
(58512825.0, 87756412.5]          0
(29269237.5, 58512825.0]          0
Name: INCOME_EQUAL_WIDTH_GROUP, dtype: int64

In [16]:
app_train['INCOME_EQUAL_FREQ_GROUP'] = pd.qcut(app_train['AMT_INCOME_TOTAL'], 4)

In [17]:
app_train['INCOME_EQUAL_FREQ_GROUP'].value_counts()
# 每個 bin 的資料筆數是一樣的

(25649.999, 112500.0]      100578
(147150.0, 202500.0]        82213
(202500.0, 117000000.0]     71538
(112500.0, 147150.0]        53182
Name: INCOME_EQUAL_FREQ_GROUP, dtype: int64

In [18]:
app_train['INCOME_CUSTOMIZED_GROUP'] = pd.cut(app_train['AMT_INCOME_TOTAL'], [-100000, 100000, 200000, 300000, 2000000])

In [20]:
app_train['INCOME_CUSTOMIZED_GROUP'].value_counts()

(100000, 200000]     155898
(200000, 300000]      65176
(-100000, 100000]     63698
(300000, 2000000]     22695
Name: INCOME_CUSTOMIZED_GROUP, dtype: int64